In [19]:
!pip install streamlit langchain openai faiss-cpu PyMuPDF python-docx
!pip install sentence-transformers


In [20]:
pip install -U langchain langchain-community faiss-cpu sentence-transformers pymupdf python-docx


Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import (
    PyMuPDFLoader,
    Docx2txtLoader,
    TextLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import display, Markdown
import os

In [22]:
# Manually specify file paths here
file_paths = [
    r"C:\Users\DAVESH KUMAR SAHU\Downloads\resume1.pdf"
]

print("Files loaded:", file_paths)


Files loaded: ['C:\\Users\\DAVESH KUMAR SAHU\\Downloads\\resume1.pdf']


In [23]:
from langchain_community.document_loaders import PyMuPDFLoader, Docx2txtLoader, TextLoader

all_docs = []

for file_path in file_paths:
    ext = os.path.splitext(file_path)[1].lower()
    
    if ext == ".pdf":
        loader = PyMuPDFLoader(file_path)
    elif ext == ".docx":
        loader = Docx2txtLoader(file_path)
    elif ext == ".txt":
        loader = TextLoader(file_path)
    else:
        print(f"[!] Unsupported file type: {file_path}")
        continue

    docs = loader.load()
    all_docs.extend(docs)

print(f"✅ Total documents loaded: {len(all_docs)}")


✅ Total documents loaded: 1


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Split into manageable text chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)

# Create embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(chunks, embedding_model)

print(f"✅ Vector DB created with {len(chunks)} chunks.")


✅ Vector DB created with 5 chunks.


In [25]:
from IPython.display import Markdown, display

query = input("🔍 Ask something about the documents: ")

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})
docs = retriever.get_relevant_documents(query)

# Display results
display(Markdown("### 📄 Top Matching Text Chunks"))
for i, doc in enumerate(docs, 1):
    display(Markdown(f"**Chunk {i}:**\n\n{doc.page_content}"))


### 📄 Top Matching Text Chunks

**Chunk 1:**

microcontrollers.
Skills
C, C++, Python, ROS2, OpenCV, Scikit-learn, TensorFlow, Embedded Systems, Arduino, Raspberry
Pi, Jetson Nano, Unity 3D, Blender, Git, Docker, Linux, IoT, Sensor Integration
Achievements & Certifications
• IEEE Idea Competition – 3rd Position for Energy Conservation (Mar 2022)
• Hackathon – AVESH, GEC Raipur (Mar 2023)
• Machine Learning Course – Udemy (Jul–Sep 2023)
• Backend Web Development – GDSC KIIT (Aug 2023)

**Chunk 2:**

Davesh Kumar Sahu
daveshkumarsahu496@gmail.com — +91 9131924504 — Raipur, Chhattisgarh, India —
linkedin.com/in/davesh-kumar-sahu — github.com/Davesh98
Summary
Dedicated Computer Science Engineering student focused on robotics, automation, and AI. Proficient
in C, C++, and Python with experience in developing autonomous mobile robots, SLAM, sensor fusion,
machine learning, and computer vision. Skilled in hardware-software integration and passionate about
building intelligent systems.
Education

**Chunk 3:**

building intelligent systems.
Education
B.Tech (Hons) in Computer Science Engineering, Chhattisgarh Swami Vivekanand Technical
University, Bhilai
2021 – 2025
Senior Secondary (PCM), Kendriya Vidyalaya No.1, Raipur
2019 – 2021
Work Experience
Embedded Systems Intern, Bot Shreyasi
Feb 2024 – Jun 2024
Engineered an autonomous robot using ROS2 for indoor navigation with LIDAR, IMU, and ultrasonic
sensors. Integrated SLAM and real-time obstacle avoidance to support automated restaurant delivery.

In [ ]:
pip install transformers accelerate


Note: you may need to restart the kernel to use updated packages.


In [29]:
from transformers import pipeline

qa_model = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=512
)


Device set to use cuda:0


In [32]:
query = "What are the phone number mentioned in this resume?"


In [33]:
# Combine top 3 chunks
context = "\n".join([doc.page_content for doc in docs])

# Format the prompt
final_prompt = f"Answer the following question based on the text below:\n\n{context}\n\nQuestion: {query}"

# Generate answer
response = qa_model(final_prompt)[0]['generated_text']

# Display
from IPython.display import Markdown, display
display(Markdown(f"### 🧠 Answer:\n\n{response}"))


### 🧠 Answer:

+91 9131924504